In [5]:
from langchain.llms import OpenAI
llm = OpenAI(temperature = 0.5) # temperature is a measure of randomness
#print(llm.predict('How old was donald trump in 1987?')) (have to register and uses openAPI credits to use)


In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (AIMessage,HumanMessage,SystemMessage)

chat = ChatOpenAI(temperature = 0.5)
#print(chat.predict_messages([HumanMessage(content="whats 9+10 (hint, the answer I want is referencing a meme)")]))

In [5]:
chat.predict("whats 9+10 (hint, the answer I want is referencing a meme)")

'21 (the answer from the "What\'s 9+10?" meme)'

In [7]:
from langchain.prompts import PromptTemplate

genericTemplate = PromptTemplate.from_template("What is the best {amenity} in {location} for {reason}")
gymPrompt = template.format(amenity = "gym", location = "Soho", reason = "athletes")
chat.predict(gymPrompt)


"As an AI language model, I do not have personal preferences or opinions. However, based on online reviews and ratings, some of the best gyms in Soho for athletes are:\n\n1. Equinox Soho: This gym offers a range of equipment and classes for athletes, including strength training, cardio, and yoga. It also has a spacious and well-maintained facility.\n\n2. Barry's Bootcamp: This gym is known for its high-intensity interval training (HIIT) classes that are designed to challenge athletes and improve their fitness levels.\n\n3. SoulCycle Soho: This gym offers indoor cycling classes that are popular among athletes looking to improve their endurance and cardiovascular health.\n\n4. Fhitting Room: This gym offers group classes that combine strength training and HIIT workouts, making it a great option for athletes looking for a challenging and varied workout.\n\n5. Rumble Boxing: This gym offers boxing-inspired workouts that are great for athletes looking to improve their strength, agility, and

**Playing with Chains**\
These pass your input first to a PromptTemplate which is then passed to an LLM. 

In [14]:
from langchain.chains import LLMChain

chain = LLMChain(llm = llm, prompt = prompt)
#chain.run({'amenity':"gym", 'location':"Soho", 'reason':"athletes"})
chain.run(amenity= "gym", location= "Soho", reason= "athletes")

'?\n\nThe best gym in Soho for athletes is Equinox Soho. It offers a wide range of equipment and classes for athletes, as well as a wide variety of amenities. It also has a team of expert trainers to help you reach your fitness goals.'

**Roles**
- Notice how in the following example, we use more specific roles (chat, system & human), rather than just the generic template we used previously (these are specifc to openAI)
- The 'System' role can be used to declare messages that determine the behaviour/ personality of the 'Assistant'. For example, your typical "You are XGPT" type of prompts. 
- The 'User' sends requests for the 'Assistant' to respond to.

**Why Chat Models?**\
The APIs are stateless, so the inputs you send are all the context the LLM will have. Chats act as a memory of sorts, collating all neccessary context you'd want based on a series of messages/prompts, e.g.:

~~~
openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
~~~

**And What about Chains?**\
Chains provide a way to link the various primitives surrounding LLMs (e.g. prompts, models and other chains)


In [2]:
from langchain.prompts.chat import (ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate)
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

GPT = ChatOpenAI(temperature=1)

system_template = "You are a linguistic guru that translates {input} to {output}"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{initial_text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt]) #input variables could be explicitly declared here but don't need to be, unlike for sequential chains
chain = LLMChain(llm=GPT, 
                 prompt = chat_prompt,
                 output_key="initial_translation") # Here we've 'chained' togeather a model and a prompt, note we didn't need to declare input variables here
chain.run(input = "Spanish",output = "English", initial_text = "Que pasa amigo?")

"Hey friend, what's up?"

**Chaining the Chains (or should i say combining the links to form a longer chain, either or, hopefully you comprende)**

In [6]:
from langchain.chains import SequentialChain

human_message_prompt_2 = HumanMessagePromptTemplate.from_template("Translate the secondary language {initial_translation} into a random latin based language")
chat_prompt_2 = ChatPromptTemplate.from_messages([human_message_prompt_2])
chain_2 = LLMChain(llm = GPT, 
                   prompt = chat_prompt_2)

combined_chain = SequentialChain(chains = [chain, chain_2], 
                                 input_variables=["input", "output", "initial_text"], # note the need for the declaration of input variables here
                                 verbose =True,)
combined_chain({"input": "Spanish", "output": "English", "initial_text": "Que pasa amigo"}) # can do this instead of chain.run



> Entering new  chain...

> Finished chain.


{'input': 'Spanish',
 'output': 'English',
 'initial_text': 'Que pasa amigo',
 'text': '"Quid novi, amice?" (in Latin)'}

**Agents**\
Agents provide autonomy in the way a workflow is undertaken. Unlike chains that use predefined steps, these use LLMs to determine their next prompt and then take action.\
Each agent needs a: tool (e.g. search engine, database lookup, etc) in order to produce a final output, LLM and name

In [2]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=1) #Use GPT as the LLM for the agent
tools = load_tools(["serpapi", "llm-math"], llm = llm)

agent = initialize_agent(tools, llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True) # Note, ZERO_SHOT_REACT_DESCRIPTION, is the default agent type anyway
agent.run("What was the peak temperature in Wandsworth London yesterday in degrees Celcius? What is that number raised to the .035 power?")



> Entering new  chain...
 I need to find out the temperature first and then raise it to a power afterwards
Action: Search
Action Input: Peak temperature in Wandsworth London yesterday
Observation: https://www.timeanddate.com/weather/@2634812/historic
Thought: It looks like the highest temperature yesterday was 19°C
Action: Calculator
Action Input: 19^0.035
Observation: Answer: 1.1085527815934937
Thought: I now know the final answer
Final Answer: 1.1085527815934937

> Finished chain.


'1.1085527815934937'

**Doing the same as above but with chat models**

In [1]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

#chat model
chat = ChatOpenAI(temperature = 1)

#tools
llm = OpenAI(temperature=1)
tools = load_tools (["serpapi", "llm-math"], llm = llm)

agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose = True)

agent.run("Produce a summary of the key players in the open source large language model ecosystem")



> Entering new  chain...
Question: Can you produce a summary of the key players in the open source large language model ecosystem?
Thought: This is an informative question, and we need to use a search engine to find the information.
Action:
```
{
  "action": "Search",
  "action_input": "key players in the open source large language model ecosystem"
}
```

Observation: Open Source Large Language Models (LLM) – Top 10 Most Powerful To Consider In 2023
Thought:This article seems promising and relevant to the question, I will read through it and take note of the key players mentioned.
Action:
```
{
  "action": "Search",
  "action_input": "https://www.analyticsinsight.net/open-source-large-language-models-llm-top-10-most-powerful-to-consider-in-2023/"
}
```

Observation: The top leading language models for NLP in 2023 are trained to understand data in text or audio format. NLP or Natural Language Processing ...
Thought:Let me continue reading and see if the article mentions specific key p

'Some of the key players in the open source large language model ecosystem include GPT-Neo, XLNet, Roberta, DeBERTa, XLM-RoBERTa, and DistilBERT.'

**Memory**\
The following example implements a simple buffer memory store, via the 'conversation' primitive

In [4]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature = 1)
conversation = ConversationChain(llm=llm, verbose = True)
conversation.run("What's up doc?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What's up doc?
AI:

> Finished chain.


" Hi there! I'm doing well. How about you?"

In [5]:
conversation.run('I am gucci doc, whats on the agenda today?')



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What's up doc?
AI:  Hi there! I'm doing well. How about you?
Human: I am gucci doc, whats on the agenda today?
AI:

> Finished chain.


" Well, I was about to review your daily calendar and take a look at what activities you have planned for today. Is there anything in particular you'd like me to focus on or help you with?"